In [1]:
%pylab inline
import pandas as pd


Populating the interactive namespace from numpy and matplotlib


In [2]:
df = pd.read_pickle('cleaned_df.pkl') 

## Preparation

## Categorical variables

In [3]:
from patsy import dmatrices


In [4]:
y, X = dmatrices('delinq ~  + loan_amnt + int_rate + installment + emp_length +'
                 'C(home_ownership) + C(grade) + C(month_issued) + C(year_issued)'
                 '+ C(purpose) + C(addr_state) + inq_last_6mths + pub_rec + revol_bal +open_acc+'
                 'collections_12_mths_ex_med + delinq_2yrs + earliest_cr_line  + fico_range_low'
                 '+ ratio_mth_inc_all_payments',
                 df, return_type='dataframe')

Earliest cr line to the year. Proabbly going to be quite good.

In [6]:
for i in X.columns:
    print(i,)

Intercept
C(home_ownership)[T.OTHER]
C(home_ownership)[T.OWN]
C(home_ownership)[T.RENT]
C(grade)[T.B]
C(grade)[T.C]
C(grade)[T.D]
C(grade)[T.E]
C(grade)[T.F]
C(grade)[T.G]
C(month_issued)[T.2]
C(month_issued)[T.3]
C(month_issued)[T.4]
C(month_issued)[T.5]
C(month_issued)[T.6]
C(month_issued)[T.7]
C(month_issued)[T.8]
C(month_issued)[T.9]
C(month_issued)[T.10]
C(month_issued)[T.11]
C(month_issued)[T.12]
C(year_issued)[T.2008]
C(year_issued)[T.2009]
C(year_issued)[T.2010]
C(year_issued)[T.2011]
C(year_issued)[T.2012]
C(year_issued)[T.2013]
C(year_issued)[T.2014]
C(purpose)[T.credit_card]
C(purpose)[T.debt_consolidation]
C(purpose)[T.educational]
C(purpose)[T.home_improvement]
C(purpose)[T.house]
C(purpose)[T.major_purchase]
C(purpose)[T.medical]
C(purpose)[T.moving]
C(purpose)[T.other]
C(purpose)[T.renewable_energy]
C(purpose)[T.small_business]
C(purpose)[T.vacation]
C(purpose)[T.wedding]
C(addr_state)[T.AL]
C(addr_state)[T.AR]
C(addr_state)[T.AZ]
C(addr_state)[T.CA]
C(addr_state)[T.CO]


In [7]:
y = np.ravel(y)

Create a small testset for testing purposes

In [ ]:
y_s, X_s = y[-10000:], X[-10000:]

## A first model: Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegressionCV

In [ ]:
model_log = LogisticRegressionCV(cv=5, penalty='l2', verbose=1, max_iter=1000)

In [ ]:
fit = model_log.fit(X_s, y_s)

In [ ]:
predictions = model_log.predict(X_s)

In [ ]:
predictions.mean()

In [ ]:
model_log.score(X_s, y_s)

In [ ]:
y_s.mean()

## Train_test_split

In [8]:
from sklearn.cross_validation import train_test_split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15,
                                                   random_state=42)

## A Second Model: 

In [10]:
from sklearn.ensemble import RandomForestClassifier

In [11]:
model_rf = RandomForestClassifier(n_estimators=50, oob_score=True, verbose=1, random_state=2143,
                                 min_samples_split=50)

In [12]:
rf_fit = model_rf.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    1.5s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:  1.3min
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:  1.3min finished


True

In [14]:
prediction = rf_fit.predict(X_test)

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:    0.7s
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.7s finished


In [15]:
prediction.mean()

0.0

In [16]:
prob_pred = rf_fit.predict_proba(X_test)

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:    0.7s
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.7s finished


In [19]:
default_probabilities = [item[1] for item in prob_pred]

In [26]:
rf_fit.oob_score_

0.89775535186069089

In [35]:
dir(range)

['__class__',
 '__contains__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'count',
 'index',
 'start',
 'step',
 'stop']

In [48]:
private_count = range(1000000)
private_count.step

1

In [69]:
def count():
    count.v = 0
    def add():
        count.v += 1
        return count.v
    return add
x = count()


In [73]:
x()

3

In [98]:
features = sorted(list(zip(X.columns, rf_fit.feature_importances_)), key=lambda x: x[1],
      reverse=True)

In [105]:
sum(feature[1] for feature in features)

0.99999999999999978

In [106]:
values = [feature[1] for feature in features]

In [119]:
100 / 4.68

21.36752136752137

In [121]:
values[0]

0.090738780260691654

## Steps

1. Standarize the values
2. multiply

In [122]:
standardized_values =(values - np.mean(values)) / np.std(values)

In [128]:
base_hundred = (100/  standardized_values[0]) * standardized_values

In [133]:
feature[0] for feature in features

('int_rate', 0.090738780260691654)

In [134]:
hundred = list(zip([feature[0] for feature in features], base_hundred))

In [135]:
hundred

[('int_rate', 100.0),
 ('ratio_mth_inc_all_payments', 99.998174004750908),
 ('revol_bal', 85.498197706312155),
 ('installment', 74.107724896520793),
 ('loan_amnt', 55.708428915496896),
 ('earliest_cr_line', 47.224535199232506),
 ('open_acc', 41.750444199661217),
 ('fico_range_low', 39.305943434182446),
 ('emp_length', 19.059672050782019),
 ('C(year_issued)[T.2014]', 15.839847040926678),
 ('inq_last_6mths', 15.805057424874123),
 ('delinq_2yrs', 6.7808707728463569),
 ('C(year_issued)[T.2012]', 1.1760744183310043),
 ('C(grade)[T.E]', 0.33011262313259815),
 ('pub_rec', -0.17174879969963144),
 ('C(grade)[T.D]', -2.0029092697161279),
 ('C(grade)[T.F]', -2.1245411402578025),
 ('C(purpose)[T.small_business]', -2.4737247603549877),
 ('C(year_issued)[T.2013]', -2.7452146777921955),
 ('C(addr_state)[T.FL]', -2.7672333546138637),
 ('C(home_ownership)[T.RENT]', -3.4472187457493146),
 ('C(grade)[T.B]', -3.4967750113322258),
 ('C(addr_state)[T.NY]', -3.6665979712625028),
 ('C(addr_state)[T.NJ]', -3.9

In [139]:
importand_featuers = hundred[:10] + hundred[-10:]

In [141]:
df.open_acc

0          3
1          3
2          2
3         10
4         15
5          9
6          7
7          4
8         11
9          2
10        14
11        12
12         4
13        11
14        14
15        11
16        14
17        20
18         4
19         8
20         8
21         7
22         8
23         6
24         7
25         8
26         7
27        14
28        12
29         8
          ..
235599    19
235600    12
235601    14
235602     4
235603    23
235604     9
235605    13
235606    19
235607    13
235608     8
235609    11
235610    11
235611    14
235612     9
235613    10
235614     9
235615    11
235616    16
235617    13
235618    10
235619    12
235620     8
235621    16
235622     9
235623     8
235624    18
235625    18
235626    21
235627    18
235628     6
Name: open_acc, dtype: float64

In [140]:
importand_featuers

[('int_rate', 100.0),
 ('ratio_mth_inc_all_payments', 99.998174004750908),
 ('revol_bal', 85.498197706312155),
 ('installment', 74.107724896520793),
 ('loan_amnt', 55.708428915496896),
 ('earliest_cr_line', 47.224535199232506),
 ('open_acc', 41.750444199661217),
 ('fico_range_low', 39.305943434182446),
 ('emp_length', 19.059672050782019),
 ('C(year_issued)[T.2014]', 15.839847040926678),
 ('C(addr_state)[T.VT]', -10.23030291883655),
 ('C(purpose)[T.renewable_energy]', -10.72564700800794),
 ('C(addr_state)[T.MS]', -10.726411523289627),
 ('C(home_ownership)[T.OTHER]', -10.911955827233541),
 ('C(purpose)[T.educational]', -11.08041049694407),
 ('C(addr_state)[T.NE]', -11.847277720791022),
 ('C(addr_state)[T.IA]', -11.882282864734762),
 ('C(addr_state)[T.ID]', -11.959292596529659),
 ('C(addr_state)[T.ME]', -11.981581551543464),
 ('Intercept', -11.981652208148629)]

In [111]:
for value in values:
    value = 

TypeError: 'function' object is not iterable

In [17]:
prob_pred.mean()

0.5

In [54]:
rf_fit.score(X_test, y_test)

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:    1.2s
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    1.2s finished


0.89873045078196867

In [ ]:
rf_full = model_rf.fit(X, y)

In [ ]:
prediction = rf_full.predict(X)

In [96]:
tuples = list(zip(y, rf_fit.predict_proba(X)))

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:    3.9s
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    3.9s finished


In [97]:
tuples

[(0.0, array([ 0.89656486,  0.10343514])),
 (1.0, array([ 0.66304684,  0.33695316])),
 (0.0, array([ 0.73805755,  0.26194245])),
 (0.0, array([ 0.83802716,  0.16197284])),
 (0.0, array([ 0.9145468,  0.0854532])),
 (0.0, array([ 0.87662657,  0.12337343])),
 (0.0, array([ 0.818842,  0.181158])),
 (0.0, array([ 0.78879857,  0.21120143])),
 (1.0, array([ 0.62657776,  0.37342224])),
 (1.0, array([ 0.66830516,  0.33169484])),
 (0.0, array([ 0.84829552,  0.15170448])),
 (0.0, array([ 0.90303472,  0.09696528])),
 (1.0, array([ 0.69287064,  0.30712936])),
 (0.0, array([ 0.80015427,  0.19984573])),
 (1.0, array([ 0.84500983,  0.15499017])),
 (0.0, array([ 0.69640697,  0.30359303])),
 (0.0, array([ 0.751328,  0.248672])),
 (0.0, array([ 0.95842638,  0.04157362])),
 (0.0, array([ 0.92821433,  0.07178567])),
 (0.0, array([ 0.96056859,  0.03943141])),
 (0.0, array([ 0.77836524,  0.22163476])),
 (1.0, array([ 0.83414748,  0.16585252])),
 (0.0, array([ 0.92152484,  0.07847516])),
 (0.0, array([ 0.8857

In [ ]:
rf_full.score(X, y)

## Support Vector Machines

In [55]:
from sklearn import svm

In [56]:
model = svm.SVC(random_state=4134)

In [ ]:
svm_fit = model.fit(X_test, y_test)

# Simple Random Forests

In [13]:
y, X = dmatrices('delinq ~ fico_range_low + pub_rec + revol_util +'
                'C(home_ownership)')

PatsyError: Error evaluating factor: NameError: name 'delinq' is not defined
    delinq ~ fico_range_low + pub_rec + revol_util +C(home_ownership)
    ^^^^^^


# Finding the average pay back in case of default

In [80]:
df.loan_status.value_counts()

Current               191303
Fully Paid            144478
Charged Off            33166
Late (31-120 days)      5748
In Grace Period         2872
Late (16-30 days)       1087
Default                  134
dtype: int64

In [84]:
df.columns

Index(['total_pymnt', 'zip_code', 'member_id', 'id', 'loan_amnt', 'int_rate',
       'installment', 'emp_length', 'home_ownership', 'grade', 'sub_grade',
       'emp_title', 'issue_d', 'loan_status', 'annual_inc',
       'verification_status', 'purpose', 'addr_state', 'inq_last_6mths', 'dti',
       'revol_util', 'mths_since_last_delinq', 'pub_rec', 'revol_bal',
       'open_acc', 'collections_12_mths_ex_med', 'delinq_2yrs',
       'earliest_cr_line', 'fico_range_low', 'last_credit_pull_d',
       'ratio_inc_debt', 'ratio_inc_installment', 'ratio_mth_inc_all_payments',
       'year_issued', 'month_issued', 'delinq'],
      dtype='object')

In [90]:
default_df = df.assign(percentage_paid= df.total_pymnt/df.loan_amnt
                      )[df.loan_status=='Charged Off']

In [92]:
default_df.percentage_paid.mean()

0.45161542363848606

# Investing only in C, D, E ,G loans with lower default probabilities